In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from pathlib import Path
from datetime import datetime
import sqlite3

In [2]:
# make query dictionary of the table name and matching querry string
query_dic = {'GRN': 'guilds-of-ravnica','WAR':'war-of-the-spark','THS': 'theros','BNG': 'born-of-the-gods','THB': 'theros-beyond-death','MID':'innistrad-midnight-hunt','10E':'10th-edition', 'BRO':'the-brothers-war', 'DGM':'dragons-maze',
 'DMU':'dominaria-united', 'ELD':'throne-of-eldraine', 'GTC':'gatecrash', 'MAT':'march-of-the-machine-the-aftermath', 
   'MOM':'march-of-the-machine', 'NEO':'kamigawa-neon-dynasty', 'ONE':'phyrexia-all-will-be-one', 'RTR':'return-to-ravnica',
  'SNC':'streets-of-new-capenna', 'VOW':'innistrad-crimson-vow', 'WOE': 'wilds-of-eldraine'}
loop_values = list(query_dic.values())  #seperate key and value
loop_key = list(query_dic.keys())   
#print(loop_values)                              ### print lists if needed
#print(loop_key)

In [3]:
browser = Browser('chrome', executable_path='../Resources/chromedriver.exe')      # set chromedriver path

In [4]:
for i in range(len(loop_key)):  # start loop in range of dictionary
    dbtable = loop_key[i]       # table to update
    url_string =  loop_values[i]   # url string related to update
    url =f"https://shop.tcgplayer.com/price-guide/magic/{url_string}"
    browser.visit(url)         
    browser.driver.maximize_window()
    time.sleep(6)                              # visit url and make soup
    req = browser.html
    soup = BeautifulSoup(req, "html")
# scrape data
    names= []
    title_tags = soup.find_all('a', class_ = "pdp-url")    # scrape card names into list
    for i in title_tags:
           names.append(i.text)
    prices =[]
    price = soup.find_all('td', class_ = "tcg-table-body__cell tcg-table-body__cell--align-right")  
    count = -1
    for i in price:                   # scrape card price to list
        count = count +1             # account for every other value in html
        if  count%2 > 0 :
            prices.append(i.text)   
    price_list= zip(names, prices)
    mkt_df = pd.DataFrame(price_list,columns = ['Name', 'Market Price'])  # create dataframe
    mkt_df['date']= datetime.today().strftime('%Y-%m-%d')
    price =  Path("../Resources/inventory.db")          # build db conection
    con = sqlite3.connect(price)
    cur = con.cursor()
    db_df = pd.read_sql_query(f"SELECT * FROM '{dbtable}'", con)  # open inventory db to df to match names
    count = 0
    while count < len(db_df):                    # while loop to take name from scraped data match to db table
        try:                                     #     if match update db if not continue loop. count put in to ensure 
            for i in range(count , len(db_df)):  #     loop continues if no match
                search = db_df.loc[i,['name']]   #
                search = search['name']
                search_df = mkt_df.loc[mkt_df['Name'] == search]
                           # break if no match
                search_df.reset_index(drop=True, inplace=True)
                search_result = search_df["Market Price"][0]
                db_df.loc[i,['MktPrice']] = search_result
                count = count + 1
        except: # if no match 
                db_df.loc[i,['MktPrice']] = "0"
                count = count + 1

    for i in range(len(mkt_df)):
        search = mkt_df.loc[i,['Name']]   # write loop df to database table and save
        search = search['Name']
        x = mkt_df.loc[i,['Market Price']]  
        x = x['Market Price']
        insert_sql = f'''UPDATE  '{dbtable}'  set MktPrice ="{x}" where "name" = "{search}"'''
        cur.execute(insert_sql)
        con.commit()                        # commit for testing
    print(f'{dbtable} updated sucesfully')  # print if updated 

GRN updated sucesfully
WAR updated sucesfully
THS updated sucesfully
BNG updated sucesfully
THB updated sucesfully
MID updated sucesfully
10E updated sucesfully
BRO updated sucesfully
DGM updated sucesfully
DMU updated sucesfully
ELD updated sucesfully
GTC updated sucesfully
MAT updated sucesfully
MOM updated sucesfully
NEO updated sucesfully
ONE updated sucesfully
RTR updated sucesfully
SNC updated sucesfully
VOW updated sucesfully
WOE updated sucesfully
